# <font color=#800517>Player Strikeout Percentage </font>


Kellie Taylor Nielson

Player data for 296 pitchers was provided from Fangraphs.com. The file contains 18 columns: 

1.Name - player’s name  
2.Team - player’s team  
3.Fangraphs_id - player’s Fangraphs ID  
4.G - player’s appearances from March 1 to July 10 2017 (the first half of the season)  
5.IP - player’s innings pitched in the first half  
6.ERA - player’s ERA in the first half  
7.FIP - player’s FIP in the first half  
8.xFIP - player’s xFIP in the first half  
9.AVG - player’s batting average against in the first half  
10.K% - player’s percentage of plate appearances that result in a strikeout in the first half  
11.BB% - player’s percentage of plate appearances that result in a walk in the first half  
12.Swing% - percentage of player’s pitches swung at in the first half  
13.Contact% - When a pitch is swung at, percentage of player’s pitches resulting in contact (# contact / # swings) in the first   half  
14.GB% - player’s percentage of balls put in play that are ground balls in the first half  
15.LD% - player’s percentage of balls in play that are line drives in the first half  
16.FB% - player’s percentage of balls in play that are fly balls  
17.2ndHalf_K% - player’s strikeout percentage for the second half of his 2017 season (7/10 - 10/1)  
18.2ndHalf_IP - player’s innings pitched for the second half of his 2017 season (7/10 - 10/1)  

A linear regression was run using the first 16 columns of data to predict a player's strikeout percentage in the 2nd half of the season. The predicted values were compared with the actual values to test the performance of the model. Simple linear regression yielded better performance results than Ridge Regression and Random Forest Regression and is therefore outlined below.

# Importing Appropriate Libraries

First, the appropriate libraries are imported.

In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


# Loading Data

The data is loaded into Jupyter notebook (in CSV format).

In [4]:
df = pd.read_csv('C:\\Users\\ktn14\\Downloads\\strikeouts.csv')

The first few rows of the dataset are displayed.

In [5]:
df.head()

,Name,Team,fangraphs_id,G,IP,ERA,FIP,xFIP,AVG,K%,BB%,Swing%,Contact%,GB%,LD%,FB%,2ndHalfK%,2ndHalfIP
0,Clayton Kershaw,Dodgers,2036,19,132.1,2.18,3.02,2.76,0.194,0.314,0.044,0.508,0.731,0.453,0.197,0.350,0.249,42.2
1,Max Scherzer,Nationals,3137,18,128.1,2.10,2.62,3.08,0.162,0.355,0.055,0.519,0.692,0.387,0.146,0.467,0.324,72.1
2,Chris Sale,Red Sox,10603,18,127.2,2.75,2.10,2.67,0.198,0.359,0.044,0.513,0.684,0.363,0.218,0.419,0.366,86.2
3,Chris Archer,Rays,6345,19,123.0,3.95,3.17,3.44,0.245,0.285,0.074,0.469,0.719,0.422,0.224,0.354,0.303,78.0
4,Ivan Nova,Pirates,1994,18,120.2,3.21,4.14,4.17,0.252,0.139,0.031,0.495,0.849,0.479,0.228,0.293,0.212,66.1


In [6]:
df.shape


(296, 18)

Simple data statistics are looked at to check for null values and data types. 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 18 columns):
Name            296 non-null object
Team            296 non-null object
fangraphs_id    296 non-null int64
G               296 non-null int64
IP              296 non-null float64
ERA             296 non-null float64
FIP             296 non-null float64
xFIP            296 non-null float64
AVG             296 non-null float64
K%              296 non-null float64
BB%             296 non-null float64
Swing%          296 non-null float64
Contact%        296 non-null float64
GB%             296 non-null float64
LD%             296 non-null float64
FB%             296 non-null float64
2ndHalfK%       296 non-null float64
2ndHalfIP       296 non-null float64
dtypes: float64(14), int64(2), object(2)
memory usage: 41.7+ KB


In [19]:
df2 = df[['G','IP', 'ERA', 'FIP', 'xFIP', 'AVG', 'K%','BB%', 'Swing%', 'Contact%', 'GB%', 'LD%', 'FB%', '2ndHalfK%', '2ndHalfIP']]
df2.head()

,G,IP,ERA,FIP,xFIP,AVG,K%,BB%,Swing%,Contact%,GB%,LD%,FB%,2ndHalfK%,2ndHalfIP
0,19,132.1,2.18,3.02,2.76,0.194,0.314,0.044,0.508,0.731,0.453,0.197,0.350,0.249,42.2
1,18,128.1,2.10,2.62,3.08,0.162,0.355,0.055,0.519,0.692,0.387,0.146,0.467,0.324,72.1
2,18,127.2,2.75,2.10,2.67,0.198,0.359,0.044,0.513,0.684,0.363,0.218,0.419,0.366,86.2
3,19,123.0,3.95,3.17,3.44,0.245,0.285,0.074,0.469,0.719,0.422,0.224,0.354,0.303,78.0
4,18,120.2,3.21,4.14,4.17,0.252,0.139,0.031,0.495,0.849,0.479,0.228,0.293,0.212,66.1


In [20]:
df2.describe()

,G,IP,ERA,FIP,xFIP,AVG,K%,BB%,Swing%,Contact%,GB%,LD%,FB%,2ndHalfK%,2ndHalfIP
count,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000,296.000000
mean,24.429054,60.499662,3.946791,4.099831,4.180203,0.239108,0.229264,0.083554,0.467956,0.768922,0.448557,0.198003,0.353463,0.225490,45.620946
std,11.280779,29.000082,1.378712,1.116321,0.828652,0.041719,0.064831,0.025188,0.034104,0.054050,0.081610,0.033911,0.081118,0.066184,24.837653
min,6.000000,30.000000,0.760000,0.790000,1.240000,0.110000,0.094000,0.015000,0.348000,0.590000,0.267000,0.081000,0.111000,0.060000,10.000000
25%,16.000000,35.200000,2.937500,3.407500,3.690000,0.209750,0.183750,0.066000,0.445000,0.732750,0.394000,0.176000,0.302750,0.182750,25.200000
50%,18.500000,46.050000,3.825000,4.065000,4.225000,0.242500,0.217000,0.082000,0.469000,0.772500,0.445500,0.198500,0.354000,0.217000,35.150000
75%,36.000000,85.400000,4.792500,4.812500,4.742500,0.264000,0.271250,0.100000,0.490000,0.810000,0.493000,0.220000,0.410000,0.266250,68.125000
max,44.000000,132.100000,8.140000,7.830000,6.110000,0.359000,0.508000,0.160000,0.576000,0.892000,0.756000,0.298000,0.580000,0.483000,110.100000


# Splitting the data into training and test sets 

The data is then split into training and test sets. inputFeatures pulls the feature variables from the dataset. inputLabel pulls the target variable from the dataset. The input features are columns 4 through 9 and 11 through 16. The target variable is column 10, the strikeout percentage in the first half of the season. Columns 1, 2, and 3 are left out of this analysis as they are simply identifiers: player name, player team, and player fangraph ID.

Train_test_split is a scikit learn utility that splits the data into training features, training target, test features, and test target. The test set size is set to 30% of the dataset. Consequently the training set is 70% of the dataset. 


In [6]:
inputFeatures = df[['G', 'IP', 'ERA', 'FIP', 'xFIP','AVG', 'BB%','Swing%','Contact%','GB%','LD%','FB%']]
inputLabel = df['K%']

trainFeatures, testFeatures, trainLabel, testLabel = train_test_split(inputFeatures,inputLabel,test_size=0.3)


# Create and Fit Model

A linear regression object is created and fit to the training data. 
The model is then used to predict the strikeout percentage.

In [7]:
lm_train = LinearRegression()
lm_train.fit(trainFeatures,trainLabel)
K_percentage =  lm_train.predict(inputFeatures)

A separate dataframe is created showing the player and predicted strikeout percentage for visualization purposes.

In [8]:
dfK = pd.DataFrame(K_percentage)
dfN = pd.DataFrame(df['Name'])
K_percentagePredict = pd.merge(dfN, dfK,left_index=True, right_index=True)
K_percentagePredict.rename(columns={0:'Predicted 2nd Half K%'}, inplace=True)


That dataframe displays a player and their predicted second half strikeout percentage. 

In [9]:
pd.set_option('display.max_rows', 500)
K_percentagePredict

,Name,Predicted 2nd Half K%
0,Clayton Kershaw,0.308306
1,Max Scherzer,0.340946
2,Chris Sale,0.349478
3,Chris Archer,0.284419
4,Ivan Nova,0.150711
5,Ervin Santana,0.190888
6,Rick Porcello,0.198553
7,Yu Darvish,0.258424
8,Jeff Samardzija,0.257710
9,Zack Greinke,0.291678


A separate dataframe is created showing the player and actual strikeout percentage in the 2nd half for visualization purposes.

In [10]:
dfA = pd.DataFrame(df['2ndHalfK%'])
K_percentageActual = pd.merge(dfN, dfA,left_index=True, right_index=True)
K_percentageActual

,Name,2ndHalfK%
0,Clayton Kershaw,0.249
1,Max Scherzer,0.324
2,Chris Sale,0.366
3,Chris Archer,0.303
4,Ivan Nova,0.212
5,Ervin Santana,0.201
6,Rick Porcello,0.200
7,Yu Darvish,0.297
8,Jeff Samardzija,0.216
9,Zack Greinke,0.244


A separate dataframe is created showing the player's predicted strikeout percentage vs actual strikeout percentage.

In [11]:
PvsA = pd.merge(K_percentagePredict,K_percentageActual, on = 'Name')
PvsA

,Name,Predicted 2nd Half K%,2ndHalfK%
0,Clayton Kershaw,0.308306,0.249
1,Max Scherzer,0.340946,0.324
2,Chris Sale,0.349478,0.366
3,Chris Archer,0.284419,0.303
4,Ivan Nova,0.150711,0.212
5,Ervin Santana,0.190888,0.201
6,Rick Porcello,0.198553,0.200
7,Yu Darvish,0.258424,0.297
8,Jeff Samardzija,0.257710,0.216
9,Zack Greinke,0.291678,0.244


# Model Performance Evaluation

The model performance compares the predicted 2nd Half strikeout percentage to the actual 2nd half strikeout percentage. 
The R squared and Mean Squared Error are calculated using metrics imported from the scikit learn library.


In [12]:
m_squared_error = mean_squared_error(dfK, dfA)
r_2_score = r2_score(dfK, dfA)

print("Mean Squared Error "+str(m_squared_error))
print("R Squared "+str(r_2_score))



Mean Squared Error 0.0028198723089647333
R Squared 0.31077040078470486


In [21]:
import re
pat = r'[A-Za-z]+'
regex = re.compile(pat)
aStr = 'Han shot first.'
print(regex.search(aStr).group(0))



Han
